In [1]:
"""
# Description:  Outputs a Microsoft Excel sheet to be analyzed with a pivot table to 
#               produce a footprint acres report as well as a point featureclass.
#
#               "Footprint" is defined as treated acres within a treatment area 
#               irespective of how many activities occured within the treatment area.
#
#               This process uses a "Speghetti and Meatballs" approach.  We first 
#               create the Spaghetti by using the "Feature to Polygon" tool.  
#               We use the "Identify" to attach ownership, vegetation, and region 
#               attributes to the Spaghetti. Then we create Meatballs using the 
#               "Feature to Points" tool.  Finally, we create the report with 
#               "Summarize Within" to find the Maximum Activity Quantity within a polygon.
#
# Known Issues: This tool under reports CalTrans footprints due to the way CalTrans 
#               reports treatment activities by road segment.
#
# Author: Spatial Informatics Group LLC
# Version: 1.0.0
# Date Created: Jan 24, 2024
"""
#TODO remove blank Categories, Counties, Veg, Region, etc. or add "Near" analysis
import os
import pandas as pd
import datetime
import arcpy
from scripts.utils import init_gdb, check_schema_lock, delete_scratch_files

workspace, scratch_workspace = init_gdb()

arcpy.EnvManager(
        workspace=workspace,
        scratchWorkspace=scratch_workspace, 
        outputCoordinateSystem= arcpy.SpatialReference("NAD 1983 California (Teale) Albers (Meters)"), #WKID 3310
        cartographicCoordinateSystem=arcpy.SpatialReference("NAD 1983 California (Teale) Albers (Meters)"), #WKID 3310
        extent="xmin=-374900, ymin=-604500, xmax=540100, ymax=450000, spatial_reference='NAD 1983 California (Teale) Albers (Meters)'", 
        preserveGlobalIds=True, 
        qualifiedFieldNames=False, 
        transferDomains=False, 
        transferGDBAttributeProperties=False, 
        overwriteOutput = True,
        )

date_id = datetime.datetime.now().strftime("%Y-%m-%d").replace('-','') # like 20221216

# INPUTS
# change path to the most updated inputs
input_poly_fc = os.path.join(workspace,'e_Appended','Treat_n_harvests_polygons_20230822')
input_ln_fc = os.path.join(workspace,'e_Appended','Treat_n_harvests_lines_20230822')
input_pt_fc = os.path.join(workspace,'e_Appended','Treat_n_harvests_points_20230822')

WFR_TF_Template = os.path.join(workspace,'b_Reference','WFR_TF_Template')
Reference_Data = os.path.join(workspace,'b_Reference','Own_Veg_Region_WUI')

# Edit where clause for footprints report
Footprints_where_clause = "(Year = 2022)" # And (Agency = 'Industrial Timber' or AGENCY = 'CALEPA' Or AGENCY = 'CALSTA' Or AGENCY = 'CNRA' Or AGENCY = 'DOD' Or AGENCY = 'DOI' Or AGENCY = 'USDA')"

# OUTPUTS
output_footprint = os.path.join(workspace,'g_Report_Data',f'Footprint_Report_2022_{date_id}')
output_footprint_pts = os.path.join(workspace,'g_Report_Data',f'Footprint_Report_2022_pts_{date_id}')
output_excel_report= os.path.join('C://Users//sageg//source//repos//mas-python//', f'Draft_Footprints_Report_Counts_to_MAS_2022_{date_id}a.xlsx')

data = arcpy.GetParameterAsText(0)

check_schema_lock(input_poly_fc)
check_schema_lock(input_ln_fc)
check_schema_lock(input_pt_fc)
check_schema_lock(output_footprint)

Available for schema lock: c:\Users\sageg\source\repos\mas-python\PC414 CWI Million Acres.gdb\e_Appended\Treat_n_harvests_polygons_20230822
Available for schema lock: c:\Users\sageg\source\repos\mas-python\PC414 CWI Million Acres.gdb\e_Appended\Treat_n_harvests_lines_20230822
Available for schema lock: c:\Users\sageg\source\repos\mas-python\PC414 CWI Million Acres.gdb\e_Appended\Treat_n_harvests_points_20230822
Available for schema lock: c:\Users\sageg\source\repos\mas-python\PC414 CWI Million Acres.gdb\g_Report_Data\Footprint_Report_2022_20230822


In [22]:
# POINTS
# Create polygons from points based on Activity Acres

#TODO add print steps
# print("     step 7/34 select layer by attribute")

Updated_Input_Table = arcpy.management.AddField(in_table=input_pt_fc, field_name="BufferMeters", field_type="DOUBLE")

Treat_n_harvests_points_20221030_2_ = arcpy.management.CalculateField(Updated_Input_Table, "BufferMeters", '"math.sqrt((!ACTIVITY_QUANTITY!*4046.86)/3.14159)"')
#TODO verify that buffer meters are being calculated

Treat_n_harvests_points_20231, Count_17_ = arcpy.management.SelectLayerByAttribute(in_layer_or_view=Treat_n_harvests_points_20221030_2_, where_clause="COUNTS_TO_MAS = 'YES'")

Output_Feature_Class_pt_ = os.path.join(scratch_workspace,'Treat_n_harvests_points_CopyFeatures')
arcpy.management.CopyFeatures(in_features=Treat_n_harvests_points_20231, out_feature_class=Output_Feature_Class_pt_)

Treat_n_harvests_point_Buffer = os.path.join(scratch_workspace,'Treat_n_harvests_point_Buffer')
arcpy.analysis.PairwiseBuffer(in_features=Output_Feature_Class_pt_, out_feature_class=Treat_n_harvests_point_Buffer, buffer_distance_or_field="BufferMeters")


<Result 'c:\\Users\\sageg\\source\\repos\\mas-python\\scratch.gdb\\Treat_n_harvests_point_Buffer'>

In [8]:
# LINES
# Create polygons from lines based on Activity Acres

Updated_Input_Table_2_ = arcpy.management.AddField(in_table=input_ln_fc, field_name="BufferMeters", field_type="DOUBLE")#[0]

Treat_n_harvests_lines_202301a, Count_2a_ = arcpy.management.SelectLayerByAttribute(in_layer_or_view=Updated_Input_Table_2_, 
    where_clause="ACTIVITY_QUANTITY > 0")

Treat_n_harvests_points_20221030_3_ = arcpy.management.CalculateField(in_table=Treat_n_harvests_lines_202301a, 
    field="BufferMeters", expression="(!ACTIVITY_QUANTITY!*4046.86)/!Shape_Length!/2")#[0]


Treat_n_harvests_lines_202301, Count_2_ = arcpy.management.SelectLayerByAttribute(in_layer_or_view=Treat_n_harvests_points_20221030_3_, 
    where_clause="COUNTS_TO_MAS = 'YES'")

Output_Feature_Class_2_ = os.path.join(scratch_workspace,'Treat_n_harvests_lines_20221115_CopyFeatures')
arcpy.management.CopyFeatures(in_features=Treat_n_harvests_lines_202301, out_feature_class=Output_Feature_Class_2_)

Treat_n_harvests_lines_202216, Count_13_ = arcpy.management.SelectLayerByAttribute(in_layer_or_view=Output_Feature_Class_2_, 
    selection_type="NEW_SELECTION", where_clause="BufferMeters >= 200 And Source = 'CalTrans'", invert_where_clause="INVERT")

Treat_n_harvests_line_Buffer_2_ = os.path.join(scratch_workspace,'Treat_n_harvests_line_Buffer')
arcpy.analysis.PairwiseBuffer(in_features=Treat_n_harvests_lines_202216, out_feature_class=Treat_n_harvests_line_Buffer_2_, 
    buffer_distance_or_field="BufferMeters") #, line_end_type="FLAT")


<Result 'c:\\Users\\sageg\\source\\repos\\mas-python\\scratch.gdb\\Treat_n_harvests_line_Buffer'>

In [9]:
# POLYGON

Treat_n_harvests_polygons_202, Count_3_ = arcpy.management.SelectLayerByAttribute(in_layer_or_view=input_poly_fc, where_clause="COUNTS_TO_MAS = 'YES'")

Treat_n_harvests_polygons_20, Count = arcpy.management.SelectLayerByAttribute(in_layer_or_view=Treat_n_harvests_polygons_202, 
    selection_type="SUBSET_SELECTION", where_clause="TREATMENT_AREA < 100000")

Treat_n_harvests_polygons_copy = os.path.join(scratch_workspace, 'Treat_n_harvests_polygons_CopyFeatures')
arcpy.management.CopyFeatures(in_features=Treat_n_harvests_polygons_20, out_feature_class=Treat_n_harvests_polygons_copy)


<Result 'c:\\Users\\sageg\\source\\repos\\mas-python\\scratch.gdb\\Treat_n_harvests_polygons_CopyFeatures'>

In [23]:
# APPEND point buffers, line buffers, and polygons

Footprints_Append = arcpy.management.CreateFeatureclass(scratch_workspace, "Footprints_Append", "POLYGON", WFR_TF_Template)


#Footprints_Append_3_ = arcpy.management.Append(inputs=[ Treat_n_harvests_polygons_copy], target=Footprints_Append, schema_type="TEST")

Footprints_Append_3_ = arcpy.management.Append(inputs=[Treat_n_harvests_point_Buffer, Treat_n_harvests_line_Buffer_2_, Treat_n_harvests_polygons_copy], target=Footprints_Append,
                                               schema_type="NO_TEST", 
                                               field_mapping="""PROJECTID_USER \"PROJECT ID USER\" true true false 40 Text 0 0,First,#,Treat_n_harvests_point_Buffer,PROJECTID_USER,0,40,Treat_n_harvests_line_Buffer_2_,PROJECTID_USER,0,40,Treat_n_harvests_polygons_copy,PROJECTID_USER,0,40;
                                               AGENCY \"AGENCY/DEPARTMENT\" true true false 55 Text 0 0,First,#,Treat_n_harvests_point_Buffer,AGENCY,0,55,Treat_n_harvests_line_Buffer_2_,AGENCY,0,55,Treat_n_harvests_polygons_copy,AGENCY,0,55;
                                               ORG_ADMIN_p \"ORG DATA STEWARD\" true true false 55 Text 0 0,First,#,Treat_n_harvests_point_Buffer,ORG_ADMIN_p,0,55,Treat_n_harvests_line_Buffer_2_,ORG_ADMIN_p,0,55,Treat_n_harvests_polygons_copy,ORG_ADMIN_p,0,55;
                                               PROJECT_CONTACT \"PROJECT CONTACT\" true true false 40 Text 0 0,First,#,Treat_n_harvests_point_Buffer,PROJECT_CONTACT,0,40,Treat_n_harvests_line_Buffer_2_,PROJECT_CONTACT,0,40,Treat_n_harvests_polygons_copy,PROJECT_CONTACT,0,40;
                                               PROJECT_EMAIL \"PROJECT EMAIL\" true true false 40 Text 0 0,First,#,Treat_n_harvests_point_Buffer,PROJECT_EMAIL,0,40,Treat_n_harvests_line_Buffer_2_,PROJECT_EMAIL,0,40,Treat_n_harvests_polygons_copy,PROJECT_EMAIL,0,40;
                                               ADMINISTERING_ORG \"ADMINISTERING ORG\" true true false 55 Text 0 0,First,#,Treat_n_harvests_point_Buffer,ADMINISTERING_ORG,0,55,Treat_n_harvests_line_Buffer_2_,ADMINISTERING_ORG,0,55,Treat_n_harvests_polygons_copy,ADMINISTERING_ORG,0,55;
                                               PROJECT_NAME \"PROJECT NAME\" true true false 125 Text 0 0,First,#,Treat_n_harvests_point_Buffer,PROJECT_NAME,0,125,Treat_n_harvests_line_Buffer_2_,PROJECT_NAME,0,125,Treat_n_harvests_polygons_copy,PROJECT_NAME,0,125;
                                               PROJECT_STATUS \"PROJECT STATUS\" true true false 25 Text 0 0,First,#,Treat_n_harvests_point_Buffer,PROJECT_STATUS,0,25,Treat_n_harvests_line_Buffer_2_,PROJECT_STATUS,0,25,Treat_n_harvests_polygons_copy,PROJECT_STATUS,0,25;
                                               PROJECT_START \"PROJECT START\" true true false 8 Date 0 0,First,#,Treat_n_harvests_point_Buffer,PROJECT_START,-1,-1,Treat_n_harvests_line_Buffer_2_,PROJECT_START,-1,-1,Treat_n_harvests_polygons_copy,PROJECT_START,-1,-1;
                                               PROJECT_END \"PROJECT END\" true true false 8 Date 0 0,First,#,Treat_n_harvests_point_Buffer,PROJECT_END,-1,-1,Treat_n_harvests_line_Buffer_2_,PROJECT_END,-1,-1,Treat_n_harvests_polygons_copy,PROJECT_END,-1,-1;
                                               PRIMARY_FUNDING_SOURCE \"PRIMARY_FUNDING_SOURCE\" true true false 130 Text 0 0,First,#,Treat_n_harvests_point_Buffer,PRIMARY_FUNDING_SOURCE,0,130,Treat_n_harvests_line_Buffer_2_,PRIMARY_FUNDING_SOURCE,0,130,Treat_n_harvests_polygons_copy,PRIMARY_FUNDING_SOURCE,0,130;
                                               PRIMARY_FUNDING_ORG \"PRIMARY_FUNDING_ORG\" true true false 130 Text 0 0,First,#,Treat_n_harvests_point_Buffer,PRIMARY_FUNDING_ORG,0,130,Treat_n_harvests_line_Buffer_2_,PRIMARY_FUNDING_ORG,0,130,Treat_n_harvests_polygons_copy,PRIMARY_FUNDING_ORG,0,130;
                                               IMPLEMENTING_ORG \"IMPLEMENTING_ORG\" true true false 55 Text 0 0,First,#,Treat_n_harvests_point_Buffer,IMPLEMENTING_ORG,0,55,Treat_n_harvests_line_Buffer_2_,IMPLEMENTING_ORG,0,55,Treat_n_harvests_polygons_copy,IMPLEMENTING_ORG,0,55;
                                               LATITUDE \"LATITUDE CENTROID\" true true false 8 Double 0 0,First,#,Treat_n_harvests_point_Buffer,LATITUDE,-1,-1,Treat_n_harvests_line_Buffer_2_,LATITUDE,-1,-1,Treat_n_harvests_polygons_copy,LATITUDE,-1,-1;
                                               LONGITUDE \"LONGITUDE CENTROID\" true true false 8 Double 0 0,First,#,Treat_n_harvests_point_Buffer,LONGITUDE,-1,-1,Treat_n_harvests_line_Buffer_2_,LONGITUDE,-1,-1,Treat_n_harvests_polygons_copy,LONGITUDE,-1,-1;
                                               BatchID_p \"Batch ID\" true true false 40 Text 0 0,First,#,Treat_n_harvests_point_Buffer,BatchID_p,0,40,Treat_n_harvests_line_Buffer_2_,BatchID_p,0,40,Treat_n_harvests_polygons_copy,BatchID_p,0,40;
                                               Val_Status_p \"Validation Status\" true true false 15 Text 0 0,First,#,Treat_n_harvests_point_Buffer,Val_Status_p,0,15,Treat_n_harvests_line_Buffer_2_,Val_Status_p,0,15,Treat_n_harvests_polygons_copy,Val_Status_p,0,15;Val_Message_p \"Validation Message\" true true false 15 Text 0 0,First,#,Treat_n_harvests_point_Buffer,Val_Message_p,0,15,Treat_n_harvests_line_Buffer_2_,Val_Message_p,0,15,Treat_n_harvests_polygons_copy,Val_Message_p,0,15;
                                               Val_RunDate_p \"Validation Run Date\" true true false 8 Date 0 0,First,#,Treat_n_harvests_point_Buffer,Val_RunDate_p,-1,-1,Treat_n_harvests_line_Buffer_2_,Val_RunDate_p,-1,-1,Treat_n_harvests_polygons_copy,Val_RunDate_p,-1,-1;Review_Status_p \"Review Status\" true true false 15 Text 0 0,First,#,Treat_n_harvests_point_Buffer,Review_Status_p,0,15,Treat_n_harvests_line_Buffer_2_,Review_Status_p,0,15,Treat_n_harvests_polygons_copy,Review_Status_p,0,15;
                                               Review_Message_p \"Review Message\" true true false 15 Text 0 0,First,#,Treat_n_harvests_point_Buffer,Review_Message_p,0,15,Treat_n_harvests_line_Buffer_2_,Review_Message_p,0,15,Treat_n_harvests_polygons_copy,Review_Message_p,0,15;
                                               Review_RunDate_p \"Review Run Date\" true true false 8 Date 0 0,First,#,Treat_n_harvests_point_Buffer,Review_RunDate_p,-1,-1,Treat_n_harvests_line_Buffer_2_,Review_RunDate_p,-1,-1,Treat_n_harvests_polygons_copy,Review_RunDate_p,-1,-1;
                                               Dataload_Status_p \"Dataload Status\" true true false 15 Text 0 0,First,#,Treat_n_harvests_point_Buffer,Dataload_Status_p,0,15,Treat_n_harvests_line_Buffer_2_,Dataload_Status_p,0,15,Treat_n_harvests_polygons_copy,Dataload_Status_p,0,15;
                                               Dataload_Msg_p \"Dataload Message\" true true false 15 Text 0 0,First,#,Treat_n_harvests_point_Buffer,Dataload_Msg_p,0,15,Treat_n_harvests_line_Buffer_2_,Dataload_Msg_p,0,15,Treat_n_harvests_polygons_copy,Dataload_Msg_p,0,15;
                                               TRMTID_USER \"TREATMENT ID USER\" true true false 40 Text 0 0,First,#,Treat_n_harvests_point_Buffer,TRMTID_USER,0,40,Treat_n_harvests_line_Buffer_2_,TRMTID_USER,0,40,Treat_n_harvests_polygons_copy,TRMTID_USER,0,40;
                                               PROJECTID \"PROJECT ID\" true true false 50 Text 0 0,First,#,Treat_n_harvests_point_Buffer,PROJECTID,0,50,Treat_n_harvests_line_Buffer_2_,PROJECTID,0,50,Treat_n_harvests_polygons_copy,PROJECTID,0,50;
                                               PROJECTNAME_ \"PROJECT NAME\" true true false 125 Text 0 0,First,#,Treat_n_harvests_point_Buffer,PROJECTNAME_,0,125,Treat_n_harvests_line_Buffer_2_,PROJECTNAME_,0,125,Treat_n_harvests_polygons_copy,PROJECTNAME_,0,125;
                                               ORG_ADMIN_t \"ORG DATA STEWARD\" true true false 255 Text 0 0,First,#,Treat_n_harvests_point_Buffer,ORG_ADMIN_t,0,255,Treat_n_harvests_line_Buffer_2_,ORG_ADMIN_t,0,255,Treat_n_harvests_polygons_copy,ORG_ADMIN_t,0,255;
                                               PRIMARY_OWNERSHIP_GROUP \"PRIMARY OWNERSHIP GROUP\" true true false 25 Text 0 0,First,#,Treat_n_harvests_point_Buffer,PRIMARY_OWNERSHIP_GROUP,0,25,Treat_n_harvests_line_Buffer_2_,PRIMARY_OWNERSHIP_GROUP,0,25,Treat_n_harvests_polygons_copy,PRIMARY_OWNERSHIP_GROUP,0,25;
                                               PRIMARY_OBJECTIVE \"PRIMARY OBJECTIVE\" true true false 65 Text 0 0,First,#,Treat_n_harvests_point_Buffer,PRIMARY_OBJECTIVE,0,65,Treat_n_harvests_line_Buffer_2_,PRIMARY_OBJECTIVE,0,65,Treat_n_harvests_polygons_copy,PRIMARY_OBJECTIVE,0,65;SECONDARY_OBJECTIVE \"SECONDARY OBJECTIVE\" true true false 65 Text 0 0,First,#,Treat_n_harvests_point_Buffer,SECONDARY_OBJECTIVE,0,65,Treat_n_harvests_line_Buffer_2_,SECONDARY_OBJECTIVE,0,65,Treat_n_harvests_polygons_copy,SECONDARY_OBJECTIVE,0,65;
                                               TERTIARY_OBJECTIVE \"TERTIARY OBJECTIVE\" true true false 65 Text 0 0,First,#,Treat_n_harvests_point_Buffer,TERTIARY_OBJECTIVE,0,65,Treat_n_harvests_line_Buffer_2_,TERTIARY_OBJECTIVE,0,65,Treat_n_harvests_polygons_copy,TERTIARY_OBJECTIVE,0,65;
                                               TREATMENT_STATUS \"TREATMENT STATUS\" true true false 25 Text 0 0,First,#,Treat_n_harvests_point_Buffer,TREATMENT_STATUS,0,25,Treat_n_harvests_line_Buffer_2_,TREATMENT_STATUS,0,25,Treat_n_harvests_polygons_copy,TREATMENT_STATUS,0,25;
                                               COUNTY \"COUNTY\" true true false 25 Text 0 0,First,#,Treat_n_harvests_point_Buffer,COUNTY,0,25,Treat_n_harvests_line_Buffer_2_,COUNTY,0,25,Treat_n_harvests_polygons_copy,COUNTY,0,25;
                                               IN_WUI \"IN WUI\" true true false 30 Text 0 0,First,#,Treat_n_harvests_point_Buffer,IN_WUI,0,30,Treat_n_harvests_line_Buffer_2_,IN_WUI,0,30,Treat_n_harvests_polygons_copy,IN_WUI,0,30;
                                               REGION \"TASK FORCE REGION\" true true false 25 Text 0 0,First,#,Treat_n_harvests_point_Buffer,REGION,0,25,Treat_n_harvests_line_Buffer_2_,REGION,0,25,Treat_n_harvests_polygons_copy,REGION,0,25;
                                               TREATMENT_AREA \"TREATMENT AREA (GIS ACRES)\" true true false 8 Double 0 0,First,#,Treat_n_harvests_point_Buffer,TREATMENT_AREA,-1,-1,Treat_n_harvests_line_Buffer_2_,TREATMENT_AREA,-1,-1,Treat_n_harvests_polygons_copy,TREATMENT_AREA,-1,-1;
                                               TREATMENT_START \"TREATMENT START\" true true false 8 Date 0 0,First,#,Treat_n_harvests_point_Buffer,TREATMENT_START,-1,-1,Treat_n_harvests_line_Buffer_2_,TREATMENT_START,-1,-1,Treat_n_harvests_polygons_copy,TREATMENT_START,-1,-1;
                                               TREATMENT_END \"TREATMENT END\" true true false 8 Date 0 0,First,#,Treat_n_harvests_point_Buffer,TREATMENT_END,-1,-1,Treat_n_harvests_line_Buffer_2_,TREATMENT_END,-1,-1,Treat_n_harvests_polygons_copy,TREATMENT_END,-1,-1;
                                               RETREATMENT_DATE_EST \"RETREATMENT DATE ESTIMATE\" true true false 8 Date 0 0,First,#,Treat_n_harvests_point_Buffer,RETREATMENT_DATE_EST,-1,-1,Treat_n_harvests_line_Buffer_2_,RETREATMENT_DATE_EST,-1,-1,Treat_n_harvests_polygons_copy,RETREATMENT_DATE_EST,-1,-1;
                                               TREATMENT_NAME \"TREATMENT NAME\" true true false 125 Text 0 0,First,#,Treat_n_harvests_point_Buffer,TREATMENT_NAME,0,125,Treat_n_harvests_line_Buffer_2_,TREATMENT_NAME,0,125,Treat_n_harvests_polygons_copy,TREATMENT_NAME,0,125;
                                               BatchID \"BATCH ID (TREATMENT)\" true true false 40 Text 0 0,First,#,Treat_n_harvests_point_Buffer,BatchID,0,40,Treat_n_harvests_line_Buffer_2_,BatchID,0,40,Treat_n_harvests_polygons_copy,BatchID,0,40;
                                               Val_Status_t \"Validation Status\" true true false 15 Text 0 0,First,#,Treat_n_harvests_point_Buffer,Val_Status_t,0,15,Treat_n_harvests_line_Buffer_2_,Val_Status_t,0,15,Treat_n_harvests_polygons_copy,Val_Status_t,0,15;
                                               Val_Message_t \"Validation Message\" true true false 15 Text 0 0,First,#,Treat_n_harvests_point_Buffer,Val_Message_t,0,15,Treat_n_harvests_line_Buffer_2_,Val_Message_t,0,15,Treat_n_harvests_polygons_copy,Val_Message_t,0,15;
                                               Val_RunDate_t \"Validation Run Date\" true true false 8 Date 0 0,First,#,Treat_n_harvests_point_Buffer,Val_RunDate_t,-1,-1,Treat_n_harvests_line_Buffer_2_,Val_RunDate_t,-1,-1,Treat_n_harvests_polygons_copy,Val_RunDate_t,-1,-1;
                                               Review_Status_t \"Review Status\" true true false 15 Text 0 0,First,#,Treat_n_harvests_point_Buffer,Review_Status_t,0,15,Treat_n_harvests_line_Buffer_2_,Review_Status_t,0,15,Treat_n_harvests_polygons_copy,Review_Status_t,0,15;
                                               Review_Message_t \"Review Message\" true true false 15 Text 0 0,First,#,Treat_n_harvests_point_Buffer,Review_Message_t,0,15,Treat_n_harvests_line_Buffer_2_,Review_Message_t,0,15,Treat_n_harvests_polygons_copy,Review_Message_t,0,15;
                                               Review_RunDate_t \"Review Run Date\" true true false 8 Date 0 0,First,#,Treat_n_harvests_point_Buffer,Review_RunDate_t,-1,-1,Treat_n_harvests_line_Buffer_2_,Review_RunDate_t,-1,-1,Treat_n_harvests_polygons_copy,Review_RunDate_t,-1,-1;
                                               Dataload_Status_t \"Dataload Status\" true true false 15 Text 0 0,First,#,Treat_n_harvests_point_Buffer,Dataload_Status_t,0,15,Treat_n_harvests_line_Buffer_2_,Dataload_Status_t,0,15,Treat_n_harvests_polygons_copy,Dataload_Status_t,0,15;
                                               Dataload_Msg_t \"Dataload Message\" true true false 15 Text 0 0,First,#,Treat_n_harvests_point_Buffer,Dataload_Msg_t,0,15,Treat_n_harvests_line_Buffer_2_,Dataload_Msg_t,0,15,Treat_n_harvests_polygons_copy,Dataload_Msg_t,0,15;
                                               ACTIVID_USER \"ACTIVITYID USER\" true true false 50 Text 0 0,First,#,Treat_n_harvests_point_Buffer,ACTIVID_USER,0,50,Treat_n_harvests_line_Buffer_2_,ACTIVID_USER,0,50,Treat_n_harvests_polygons_copy,ACTIVID_USER,0,50;
                                               TREATMENTID_ \"TREATMENTID\" true true false 50 Text 0 0,First,#,Treat_n_harvests_point_Buffer,TREATMENTID_,0,50,Treat_n_harvests_line_Buffer_2_,TREATMENTID_,0,50,Treat_n_harvests_polygons_copy,TREATMENTID_,0,50;
                                               ORG_ADMIN_a \"ORG DATA STEWARD\" true true false 150 Text 0 0,First,#,Treat_n_harvests_point_Buffer,ORG_ADMIN_a,0,150,Treat_n_harvests_line_Buffer_2_,ORG_ADMIN_a,0,150,Treat_n_harvests_polygons_copy,ORG_ADMIN_a,0,150;
                                               ACTIVITY_DESCRIPTION \"ACTIVITY DESCRIPTION\" true true false 70 Text 0 0,First,#,Treat_n_harvests_point_Buffer,ACTIVITY_DESCRIPTION,0,70,Treat_n_harvests_line_Buffer_2_,ACTIVITY_DESCRIPTION,0,70,Treat_n_harvests_polygons_copy,ACTIVITY_DESCRIPTION,0,70;
                                               ACTIVITY_CAT \"ACTIVITY CATEGORY\" true true false 40 Text 0 0,First,#,Treat_n_harvests_point_Buffer,ACTIVITY_CAT,0,40,Treat_n_harvests_line_Buffer_2_,ACTIVITY_CAT,0,40,Treat_n_harvests_polygons_copy,ACTIVITY_CAT,0,40;
                                               BROAD_VEGETATION_TYPE \"BROAD VEGETATION TYPE\" true true false 50 Text 0 0,First,#,Treat_n_harvests_point_Buffer,BROAD_VEGETATION_TYPE,0,50,Treat_n_harvests_line_Buffer_2_,BROAD_VEGETATION_TYPE,0,50,Treat_n_harvests_polygons_copy,BROAD_VEGETATION_TYPE,0,50;
                                               BVT_USERD \"IS BVT USER DEFINED\" true true false 3 Text 0 0,First,#,Treat_n_harvests_point_Buffer,BVT_USERD,0,3,Treat_n_harvests_line_Buffer_2_,BVT_USERD,0,3,Treat_n_harvests_polygons_copy,BVT_USERD,0,3;
                                               ACTIVITY_STATUS \"ACTIVITY STATUS\" true true false 25 Text 0 0,First,#,Treat_n_harvests_point_Buffer,ACTIVITY_STATUS,0,25,Treat_n_harvests_line_Buffer_2_,ACTIVITY_STATUS,0,25,Treat_n_harvests_polygons_copy,ACTIVITY_STATUS,0,25;
                                               ACTIVITY_QUANTITY \"ACTIVITY QUANTITY\" true true false 8 Double 0 0,First,#,Treat_n_harvests_point_Buffer,ACTIVITY_QUANTITY,-1,-1,Treat_n_harvests_line_Buffer_2_,ACTIVITY_QUANTITY,-1,-1,Treat_n_harvests_polygons_copy,ACTIVITY_QUANTITY,-1,-1;
                                               ACTIVITY_UOM \"ACTIVITY UNITS\" true true false 15 Text 0 0,First,#,Treat_n_harvests_point_Buffer,ACTIVITY_UOM,0,15,Treat_n_harvests_line_Buffer_2_,ACTIVITY_UOM,0,15,Treat_n_harvests_polygons_copy,ACTIVITY_UOM,0,15;
                                               ACTIVITY_START \"ACTIVITY START\" true true false 8 Date 0 0,First,#,Treat_n_harvests_point_Buffer,ACTIVITY_START,-1,-1,Treat_n_harvests_line_Buffer_2_,ACTIVITY_START,-1,-1,Treat_n_harvests_polygons_copy,ACTIVITY_START,-1,-1;
                                               ACTIVITY_END \"ACTIVITY END\" true true false 8 Date 0 0,First,#,Treat_n_harvests_point_Buffer,ACTIVITY_END,-1,-1,Treat_n_harvests_line_Buffer_2_,ACTIVITY_END,-1,-1,Treat_n_harvests_polygons_copy,ACTIVITY_END,-1,-1;
                                               ADMIN_ORG_NAME \"ADMINISTRATION ORGANIZATION NAME\" true true false 150 Text 0 0,First,#,Treat_n_harvests_point_Buffer,ADMIN_ORG_NAME,0,150,Treat_n_harvests_line_Buffer_2_,ADMIN_ORG_NAME,0,150,Treat_n_harvests_polygons_copy,ADMIN_ORG_NAME,0,150;IMPLEM_ORG_NAME \"IMPLEMENTATION ORGANIZATION NAME\" true true false 150 Text 0 0,First,#,Treat_n_harvests_point_Buffer,IMPLEM_ORG_NAME,0,150,Treat_n_harvests_line_Buffer_2_,IMPLEM_ORG_NAME,0,150,Treat_n_harvests_polygons_copy,IMPLEM_ORG_NAME,0,150;
                                               PRIMARY_FUND_SRC_NAME \"PRIMARY FUND SOURCE NAME\" true true false 100 Text 0 0,First,#,Treat_n_harvests_point_Buffer,PRIMARY_FUND_SRC_NAME,0,100,Treat_n_harvests_line_Buffer_2_,PRIMARY_FUND_SRC_NAME,0,100,Treat_n_harvests_polygons_copy,PRIMARY_FUND_SRC_NAME,0,100;
                                               PRIMARY_FUND_ORG_NAME \"PRIMARY FUND ORGANIZATION NAME\" true true false 100 Text 0 0,First,#,Treat_n_harvests_point_Buffer,PRIMARY_FUND_ORG_NAME,0,100,Treat_n_harvests_line_Buffer_2_,PRIMARY_FUND_ORG_NAME,0,100,Treat_n_harvests_polygons_copy,PRIMARY_FUND_ORG_NAME,0,100;
                                               SECONDARY_FUND_SRC_NAME \"SECONDARY FUND SOURCE NAME\" true true false 100 Text 0 0,First,#,Treat_n_harvests_point_Buffer,SECONDARY_FUND_SRC_NAME,0,100,Treat_n_harvests_line_Buffer_2_,SECONDARY_FUND_SRC_NAME,0,100,Treat_n_harvests_polygons_copy,SECONDARY_FUND_SRC_NAME,0,100;
                                               SECONDARY_FUND_ORG_NAME \"SECONDARY FUND ORGANIZATION NAME\" true true false 100 Text 0 0,First,#,Treat_n_harvests_point_Buffer,SECONDARY_FUND_ORG_NAME,0,100,Treat_n_harvests_line_Buffer_2_,SECONDARY_FUND_ORG_NAME,0,100,Treat_n_harvests_polygons_copy,SECONDARY_FUND_ORG_NAME,0,100;
                                               TERTIARY_FUND_SRC_NAME \"TERTIARY FUND SOURCE NAME\" true true false 100 Text 0 0,First,#,Treat_n_harvests_point_Buffer,TERTIARY_FUND_SRC_NAME,0,100,Treat_n_harvests_line_Buffer_2_,TERTIARY_FUND_SRC_NAME,0,100,Treat_n_harvests_polygons_copy,TERTIARY_FUND_SRC_NAME,0,100;
                                               TERTIARY_FUND_ORG_NAME \"TERTIARY FUND ORGANIZATION NAME\" true true false 100 Text 0 0,First,#,Treat_n_harvests_point_Buffer,TERTIARY_FUND_ORG_NAME,0,100,Treat_n_harvests_line_Buffer_2_,TERTIARY_FUND_ORG_NAME,0,100,Treat_n_harvests_polygons_copy,TERTIARY_FUND_ORG_NAME,0,100;
                                               ACTIVITY_PRCT \"ACTIVITY PERCENT\" true true false 8 Double 0 0,First,#,Treat_n_harvests_point_Buffer,ACTIVITY_PRCT,-1,-1,Treat_n_harvests_line_Buffer_2_,ACTIVITY_PRCT,-1,-1,Treat_n_harvests_polygons_copy,ACTIVITY_PRCT,-1,-1;
                                               RESIDUE_FATE \"RESIDUE FATE\" true true false 35 Text 0 0,First,#,Treat_n_harvests_point_Buffer,RESIDUE_FATE,0,35,Treat_n_harvests_line_Buffer_2_,RESIDUE_FATE,0,35,Treat_n_harvests_polygons_copy,RESIDUE_FATE,0,35;
                                               RESIDUE_FATE_QUANTITY \"RESIDUE FATE QUANTITY\" true true false 8 Double 0 0,First,#,Treat_n_harvests_point_Buffer,RESIDUE_FATE_QUANTITY,-1,-1,Treat_n_harvests_line_Buffer_2_,RESIDUE_FATE_QUANTITY,-1,-1,Treat_n_harvests_polygons_copy,RESIDUE_FATE_QUANTITY,-1,-1;
                                               RESIDUE_FATE_UNITS \"RESIDUE FATE UNITS\" true true false 5 Text 0 0,First,#,Treat_n_harvests_point_Buffer,RESIDUE_FATE_UNITS,0,5,Treat_n_harvests_line_Buffer_2_,RESIDUE_FATE_UNITS,0,5,Treat_n_harvests_polygons_copy,RESIDUE_FATE_UNITS,0,5;
                                               ACTIVITY_NAME \"ACTIVITY NAME\" true true false 150 Text 0 0,First,#,Treat_n_harvests_point_Buffer,ACTIVITY_NAME,0,150,Treat_n_harvests_line_Buffer_2_,ACTIVITY_NAME,0,150,Treat_n_harvests_polygons_copy,ACTIVITY_NAME,0,150;VAL_STATUS_a \"VALIDATION STATUS\" true true false 15 Text 0 0,First,#,Treat_n_harvests_point_Buffer,VAL_STATUS_a,0,15,Treat_n_harvests_line_Buffer_2_,VAL_STATUS_a,0,15,Treat_n_harvests_polygons_copy,VAL_STATUS_a,0,15;
                                               VAL_MSG_a \"VALIDATION MESSAGE\" true true false 15 Text 0 0,First,#,Treat_n_harvests_point_Buffer,VAL_MSG_a,0,15,Treat_n_harvests_line_Buffer_2_,VAL_MSG_a,0,15,Treat_n_harvests_polygons_copy,VAL_MSG_a,0,15;
                                               VAL_RUNDATE_a \"VALIDATION RUN DATE\" true true false 8 Date 0 0,First,#,Treat_n_harvests_point_Buffer,VAL_RUNDATE_a,-1,-1,Treat_n_harvests_line_Buffer_2_,VAL_RUNDATE_a,-1,-1,Treat_n_harvests_polygons_copy,VAL_RUNDATE_a,-1,-1;
                                               REVIEW_STATUS_a \"REVIEW STATUS\" true true false 15 Text 0 0,First,#,Treat_n_harvests_point_Buffer,REVIEW_STATUS_a,0,15,Treat_n_harvests_line_Buffer_2_,REVIEW_STATUS_a,0,15,Treat_n_harvests_polygons_copy,REVIEW_STATUS_a,0,15;
                                               REVIEW_MSG_a \"REVIEW MESSAGE\" true true false 15 Text 0 0,First,#,Treat_n_harvests_point_Buffer,REVIEW_MSG_a,0,15,Treat_n_harvests_line_Buffer_2_,REVIEW_MSG_a,0,15,Treat_n_harvests_polygons_copy,REVIEW_MSG_a,0,15;
                                               REVIEW_RUNDATE_a \"REVIEW RUN DATE\" true true false 8 Date 0 0,First,#,Treat_n_harvests_point_Buffer,REVIEW_RUNDATE_a,-1,-1,Treat_n_harvests_line_Buffer_2_,REVIEW_RUNDATE_a,-1,-1,Treat_n_harvests_polygons_copy,REVIEW_RUNDATE_a,-1,-1;
                                               DATALOAD_STATUS_a \"DATALOAD STATUS\" true true false 15 Text 0 0,First,#,Treat_n_harvests_point_Buffer,DATALOAD_STATUS_a,0,15,Treat_n_harvests_line_Buffer_2_,DATALOAD_STATUS_a,0,15,Treat_n_harvests_polygons_copy,DATALOAD_STATUS_a,0,15;
                                               DATALOAD_MSG_a \"DATALOAD MESSAGE\" true true false 15 Text 0 0,First,#,Treat_n_harvests_point_Buffer,DATALOAD_MSG_a,0,15,Treat_n_harvests_line_Buffer_2_,DATALOAD_MSG_a,0,15,Treat_n_harvests_polygons_copy,DATALOAD_MSG_a,0,15;
                                               Source \"Source\" true true false 65 Text 0 0,First,#,Treat_n_harvests_point_Buffer,Source,0,65,Treat_n_harvests_line_Buffer_2_,Source,0,65,Treat_n_harvests_polygons_copy,Source,0,65;
                                               Year \"Calendar Year\" true true false 4 Long 0 0,First,#,Treat_n_harvests_point_Buffer,Year,-1,-1,Treat_n_harvests_line_Buffer_2_,Year,-1,-1,Treat_n_harvests_polygons_copy,Year,-1,-1;
                                               Year_txt \"Year as Text\" true true false 255 Text 0 0,First,#,Treat_n_harvests_point_Buffer,Year_txt,0,255,Treat_n_harvests_line_Buffer_2_,Year_txt,0,255,Treat_n_harvests_polygons_copy,Year_txt,0,255;
                                               Act_Code \"USFS Activity Code\" true true false 4 Long 0 0,First,#,Treat_n_harvests_point_Buffer,Act_Code,-1,-1,Treat_n_harvests_line_Buffer_2_,Act_Code,-1,-1,Treat_n_harvests_polygons_copy,Act_Code,-1,-1;
                                               Crosswalk \"Crosswalk Activities\" true true false 150 Text 0 0,First,#,Treat_n_harvests_point_Buffer,Crosswalk,0,150,Treat_n_harvests_line_Buffer_2_,Crosswalk,0,150,Treat_n_harvests_polygons_copy,Crosswalk,0,150;
                                               Federal_FY \"Federal FY\" true true false 4 Long 0 0,First,#,Treat_n_harvests_point_Buffer,Federal_FY,-1,-1,Treat_n_harvests_line_Buffer_2_,Federal_FY,-1,-1,Treat_n_harvests_polygons_copy,Federal_FY,-1,-1;
                                               State_FY \"State FY\" true true false 4 Long 0 0,First,#,Treat_n_harvests_point_Buffer,State_FY,-1,-1,Treat_n_harvests_line_Buffer_2_,State_FY,-1,-1,Treat_n_harvests_polygons_copy,State_FY,-1,-1"""
                                               )

fc = arcpy.GetParameterAsText(0)
arcpy.GetCount_management(Footprints_Append_3_)
# Print all of the geoprocessing messages returned by the
#  last tool (GetCount)
print(arcpy.GetMessages())

NameError: name 'Treat_n_harvests_line_Buffer_2_' is not defined

In [11]:

# Process: Select By Year and Agency (Select) (analysis)
# This Feature Class is used to create both the Spaghetti and the Meatballs

Footprints_Append_Where = os.path.join(scratch_workspace, 'Footprints_Append_Where')

arcpy.analysis.Select(in_features=Footprints_Append, out_feature_class=Footprints_Append_Where, 
    where_clause=Footprints_where_clause)
    
result=arcpy.management.GetCount(Footprints_Append_Where)
print('{} has {} records'.format(Footprints_Append_Where, result[0]))



c:\Users\sageg\source\repos\mas-python\scratch.gdb\Footprints_Append_Where has 31073 records


In [12]:
# Process: Repair Geometry (Repair Geometry) (management)

#Repaired_Input_Features = arcpy.management.RepairGeometry(in_features=Footprints_Append_Where)

#NOTE Last Run took 24hrs.  Try running without repair. 

In [13]:
#
# Create Meatballs
#

# Process: Feature To Point (3) (Feature To Point) (management)
Meatballs = os.path.join(scratch_workspace, 'Meatballs')

arcpy.management.FeatureToPoint(in_features=Footprints_Append_Where, out_feature_class=Meatballs, point_location="INSIDE")

#
# Create the Spaghetti
#

# Process: PairwiseDissolve (Analysis)
Spaghetti_Dissolve = os.path.join(scratch_workspace, 'Spaghetti_Dissolve')
arcpy.analysis.PairwiseDissolve(Footprints_Append_Where, Spaghetti_Dissolve, ["TRMTID_USER"])

#NOTE Before CalTrans Treatment ID change Run took 5.5hrs.
# Last run took 3min

<Result 'c:\\Users\\sageg\\source\\repos\\mas-python\\scratch.gdb\\Spaghetti_Dissolve'>

In [14]:

# Process: Feature To Polygon (Feature To Polygon) (management)
Spaghetti_FeatureToPolygon = os.path.join(scratch_workspace, 'Spaghetti_FeatureToPolygon')

arcpy.management.FeatureToPolygon(in_features=[Spaghetti_Dissolve], out_feature_class=Spaghetti_FeatureToPolygon)
result=arcpy.management.GetCount(Spaghetti_FeatureToPolygon)
print('{} has {} records'.format(Spaghetti_FeatureToPolygon, result[0]))

#NOTE Last Run took 8hrs.
#NOTE Before CalTrans Treatment ID change Run took 6hrs.
#NOTE Last Run took 41s.

c:\Users\sageg\source\repos\mas-python\scratch.gdb\Spaghetti_FeatureToPolygon has 38951 records


In [15]:

# Process: Delete Field (2) (Delete Field) (management)

Spaghetti_Delete_Fields = arcpy.management.DeleteField(in_table=Spaghetti_FeatureToPolygon, 
    drop_field=["FID_Spaghetti_Dissolve", "TRMTID_USER"])
    

In [16]:

# Add Sauce: Ownership, Vegetation, and Region attributes to the Spaghetti
Sauce = Reference_Data

# Process: Identity (9) (Identity) (analysis)
Spaghetti_n_Sauce = os.path.join(scratch_workspace, 'Spaghetti_n_Sauce')

arcpy.analysis.Identity(in_features=Spaghetti_Delete_Fields, identity_features=Sauce, 
    out_feature_class=Spaghetti_n_Sauce, join_attributes="NO_FID")

#NOTE Before CalTrans Treatment ID change Run took 43 minutes.
#NOTE Last Run took 8min.

<Result 'c:\\Users\\sageg\\source\\repos\\mas-python\\scratch.gdb\\Spaghetti_n_Sauce'>

In [17]:
# Ensure the ownership for CalTrans projects are State

# Process: Select CalTrans (3) (Select) (analysis)
CalTrans_Projects = os.path.join(scratch_workspace, 'CalTrans_Projects')

arcpy.analysis.Select(in_features=Footprints_Append, out_feature_class=CalTrans_Projects, 
    where_clause="AGENCY = 'CALSTA'")

# Process: Pairwise Dissolve (6) (Pairwise Dissolve) (analysis)
CalTrans_Projects_Dissolve = os.path.join(scratch_workspace, 'CalTrans_Projects_Dissolve')

arcpy.analysis.PairwiseDissolve(in_features=CalTrans_Projects, out_feature_class=CalTrans_Projects_Dissolve, 
    dissolve_field=["AGENCY"])

#NOTE Last Run took 1.5min.

<Result 'c:\\Users\\sageg\\source\\repos\\mas-python\\scratch.gdb\\CalTrans_Projects_Dissolve'>

In [20]:

# Process: Select Layer by Location (Data Management) - Select CalTrans Projects to set Ownership to State
Select_Spaghetti_by_CalTrans1 = arcpy.management.SelectLayerByLocation(Spaghetti_n_Sauce, "HAVE_THEIR_CENTER_IN", CalTrans_Projects_Dissolve, 10, "NEW_SELECTION")


In [ ]:

# Process: Calculate CalTrans = State (Calculate Field) (management)
Select_Spaghetti_by_CalTrans2 = arcpy.management.CalculateField(in_table=Select_Spaghetti_by_CalTrans1, field="PRIMARY_OWNERSHIP_GROUP", 
    expression="\"STATE\"")


In [ ]:

# Process: Clear Selection (Select Layer By Attribute) (management)
Select_Spaghetti_by_CalTrans3, Count_29_ = arcpy.management.SelectLayerByAttribute(in_layer_or_view=Select_Spaghetti_by_CalTrans2, 
    selection_type="CLEAR_SELECTION")

#NOTE Last Run took 2.5hrs.

In [2]:

# Process: Add Field Footprint Acres (Add Field) (management)

Treat_n_harvests_po_Identity1_8_ = arcpy.management.AddField(in_table=Select_Spaghetti_by_CalTrans3, 
                                                             field_name="FootprintAcres", field_type="DOUBLE", field_precision=8, field_scale=1)

# Process: Calculate Geometry Attributes (4) (Calculate Geometry Attributes) (management)

Spaghetti2 = arcpy.management.CalculateGeometryAttributes(in_features=Treat_n_harvests_po_Identity1_8_, 
                                                                                geometry_property=[["FootprintAcres", "AREA"]], area_unit="ACRES_US")


NameError: name 'Select_Spaghetti_by_CalTrans3' is not defined

In [28]:
#
# Make Dinner
#

Dinner = os.path.join(scratch_workspace, 'Dinner')

arcpy.DefineProjection_management(Spaghetti_n_Sauce, arcpy.SpatialReference("NAD 1983 California (Teale) Albers (Meters)"))
arcpy.DefineProjection_management(Meatballs, arcpy.SpatialReference("NAD 1983 California (Teale) Albers (Meters)"))

# Process: Summarize Within (2) (Summarize Within) (analysis)
arcpy.analysis.SummarizeWithin(in_polygons=Spaghetti_n_Sauce, 
                                            in_sum_features=Meatballs, 
                                            out_feature_class=Dinner,
                                            keep_all_polygons="ONLY_INTERSECTING",
                                            sum_fields=[["ACTIVITY_QUANTITY", "Sum"], ["ACTIVITY_QUANTITY", "Mean"], ["ACTIVITY_QUANTITY", "MIn"], ["ACTIVITY_QUANTITY", "Max"]])
#NOTE Last Run took 3min.

<Result 'c:\\Users\\sageg\\source\\repos\\mas-python\\scratch.gdb\\Dinner'>

In [29]:

# Process: Select (Select) (analysis)
arcpy.analysis.Select(in_features=Dinner, out_feature_class=output_footprint)

# Process: Assign Domain To Field (Data Management)
arcpy.AssignDomainToField_management(output_footprint, "PRIMARY_OWNERSHIP_GROUP", "D_PR_OWN_GR")
arcpy.AssignDomainToField_management(output_footprint, "COUNTY", "D_CNTY")
arcpy.AssignDomainToField_management(output_footprint, "BROAD_VEGETATION_TYPE", "D_BVT")
arcpy.AssignDomainToField_management(output_footprint, "REGION", "D_TASKFORCE")


<Result 'c:\\Users\\sageg\\source\\repos\\mas-python\\PC414 CWI Million Acres.gdb\\g_Report_Data\\Footprint_Report_2022_20230822'>

In [30]:

# Process: Feature To Point (2) (Feature To Point) (management)
Footprint_Report_2021_2022_pts = os.path.join(workspace, 'g_Report_Data', output_footprint)
#if Footprint_Report_pts_20230720_lyrx and Treat_n_harvests_point_Buffer and Value_3_:
arcpy.management.FeatureToPoint(in_features=output_footprint, 
    out_feature_class=output_footprint_pts, point_location="INSIDE")

# Process: Assign Domain To Field (Data Management)
arcpy.AssignDomainToField_management(output_footprint_pts, "PRIMARY_OWNERSHIP_GROUP", "D_PR_OWN_GR")
arcpy.AssignDomainToField_management(output_footprint_pts, "COUNTY", "D_CNTY")
arcpy.AssignDomainToField_management(output_footprint_pts, "BROAD_VEGETATION_TYPE", "D_BVT")
arcpy.AssignDomainToField_management(output_footprint_pts, "REGION", "D_TASKFORCE")

arcpy.management.DeleteField(in_table=output_footprint_pts, drop_field=["ORIG_FID"])


<Result 'c:\\Users\\sageg\\source\\repos\\mas-python\\PC414 CWI Million Acres.gdb\\g_Report_Data\\Footprint_Report_2022_pts_20230822'>

In [31]:

# Process: Table To Excel (4) (Table To Excel) (conversion)

arcpy.conversion.TableToExcel([output_footprint], 
                                output_excel_report, 
                                "ALIAS", 
                                "DESCRIPTION")


<Result 'C:\\\\Users\\\\sageg\\\\source\\\\repos\\\\mas-python\\Draft_Footprints_Report_Counts_to_MAS_2022_20230822a.xlsx'>

In [32]:
#TODO add pivot table

# df = pd.read_excel(output_excel_report2)

# pivot1 = df.pivot_table(index = ["REGION"], 
#                 values="Maximum ACTIVITY_QUANTITY",
#                 aggfunc = 'sum', 
#                 margins=True, 
#                 margins_name='Total')
# pos_neg_fmt = pivot1.add_format({'num_format': '#,##0;(#,##0)'})
# df.style.format(precision=0)
# pivot1.add_format({'num_format':'#,##0'})
# print(pivot1)


In [33]:
# pivot1.to_excel(output_excel_report2, 'Region')


In [34]:
# from xlsxwriter import Workbook
# writer = pd.ExcelWriter(output_excel_report2) #, engine='xlsxwriter'
# # pivot.to_excel(writer, sheet_name='Region', index=True)
# df.to_excel(writer,'Region')
# workbook = writer.book
# worksheet = writer.sheets['Region']
# # format1 = workbook.add_format({'num_format': '#,##0'})
# workbook.number_format = '#,##0'
# # worksheet.set_column('B', 18, format1)
# writer.save()

In [35]:
# print('   Deleting Scratch Files')
# delete_scratch_files(gdb = scratch_workspace, delete_fc = 'yes', delete_table = 'yes', delete_ds = 'yes')